<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/DL/Group_3DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning with PyTorch
## 3D Convolutional Neural Network on Group Brain fMRI
Contributors: Stacey Rivet Beck, Ben Merrill
### To Do:
- Either:
  - Get Raw data in 4D   
          - OR -
  - Reshape Raw data into 4D from 2D
    - Apply Whole Brain Mask to data and save to AWS

- Build Dataloader: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

- Implement 3DCNN from paper: REALLY GREAT PAPERS
  - Nguyen et al. http://proceedings.mlr.press/v136/nguyen20a/nguyen20a.pdf
  - Wang et al. https://arxiv.org/pdf/1801.09858.pdf (discusses more in detail the input data shapes and processing)
  
  - Inputs: 84@ x * y * z ; one fmri time series at a time, not concatenated
  - Basic Architecture:
        #First layer is generating temporal descriptors of the voxels
        Conv1 1 x 1 x 1 filter, output = 32, stride = 1, ReLU, BatchNorm
        Conv2 7 x 7 x 7 filter, output = 64, stride = 2, ReLU, BatchNorm
        Conv3 3 x 3 x 3 fitler, output = 64, stride = 2, ReLU, BatchNorm
        Conv4 3 x 3 x 3 fitler, output = 128, stride = 2, ReLU, BatchNorm
        Global Average Pooling on final feature maps ->
        Flattened maps size 128?
        Fully connected layer size 64
        Fully connected layer size 2 (2 way classification, one for each class) -> softmax

        Optimized with Adam, standard parameters (β1=0.9 and β2=0.999)
        Batched at 32, but we may need to batch smaller due to GPU compute
        Learning Rate = 0.001, gradual decay after Val loss plateaued after 15 epochs
        Cross entropy Loss
        Employ early stopping
        In Wang et al. they used data for visualization, same size as input data, though are reduced in time dimension to be mapped on fsaverage surface. 
        
  

## Importing Dataset and Labels

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')  

Mounted at /content/gdrive


In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source/DL
!ls

Cloning into 'teambrainiac'...
remote: Enumerating objects: 1005, done.
remote: Counting objects: 100% (1005/1005), done.
remote: Compressing objects: 100% (766/766), done.
remote: Total 1005 (delta 639), reused 437 (delta 223), pack-reused 0
Receiving objects: 100% (1005/1005), 77.13 MiB | 3.21 MiB/s, done.
Resolving deltas: 100% (639/639), done.
/content/teambrainiac/source/DL
dataloader_class.py  Group_3DCNN.ipynb	process_dl.py  utils_dl.py


### Load path_config.py

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 196 bytes


## Import Packages

In [17]:
# Possible Missing Packages
# !pip install boto3

In [5]:
# General Library Imports
import re
import scipy.io
import os
import pickle
import numpy as np
import nibabel as nib
import pandas as pd
import boto3
import tempfile
import tqdm
from path_config import mat_path
from botocore.exceptions import ClientError
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

# From Local Directory
from utils_dl import *
from process_dl import *

# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

#import torchvision.transforms as transforms
from torch.nn import ReLU, CrossEntropyLoss, Conv3d, Module, Softmax, AdaptiveAvgPool3d
from torch.optim import Adam, SGD

#from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
# from dataloader_class import DatasetFmri


## Import Group fMRI Data, Normalize, and Create Masks

In [6]:
# Open path dictionary file to get subject ids
path = "../data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)

In [7]:
# Fix hyperparameters for preprocessing
label_type='rt_labels'
n_subjects = 1
runs_list = [2,3,4]


# Run functions to import unmasked data
image_label_mask, image_labels = labels_mask_binary(data_path_dict, label_type='rt_labels')

one_subject = load_subjects_chronologically(data_path_dict, n_subjects, image_label_mask, image_labels, label_type, runs_list)

Subject ids loaded.
Adding subjects to dictionary.


1it [00:15, 15.09s/it]


In [8]:
# Get Mask Indicies. Note that 'mask' is the string for a whole brain mask.
whole_brain_mask = get_mask('mask', data_path_dict, mask_ind=0)
scaler = 'standard'

fully_normalized_subject = mask_normalize_runs_reshape_3d(one_subject, whole_brain_mask, scaler)

Completed Subject 0


In [9]:
fully_normalized_subject['10004_08693']['run_2'].shape

(84, 79, 95, 79)

## Format Train and Test Data for Single Subject

In [134]:
# Add In-Channel Layer (Needed for imput to pytorch functions)

def add_in_channel(t_dict):
  '''
  Adds an additional layer to training or testing dict as the 'in_channel' parameter for a CNN in pytorch
  '''

  images = []
  dict_with_inchannel = {}

  for image in t_dict['images']:
    in_channel_image = np.array([image])
    images.append(in_channel_image)
  
  dict_with_inchannel['images'] = np.array(images)
  dict_with_inchannel['labels'] = t_dict['labels']

  return dict_with_inchannel


train_dict = add_in_channel(train_dict)

In [135]:
train_runs = [2]
test_runs = [3,4]
subject = fully_normalized_subject['10004_08693']

train_dict, test_dict = train_test_aggregation(subject, train_runs, test_runs)

train_dict = add_in_channel(train_dict)

Train Runs Done
Test Runs Done


## Build Dataloader

In [136]:
# train_dict['labels'][3]

In [137]:
# from torch.utils.data import Dataset, DataLoader

# class DatasetFmri(Dataset):
#     def __init__(self, image_dictionary, transform=None, target_transform=None):
#         self.labels = image_dictionary['labels']
#         self.images = torch.from_numpy(image_dictionary['images'])
#         # Maybe set up transfers later

#     def __len__(self):
#         return len(self.img_labels)

#     def __getitem__(self, idx):
#         image = self.images[idx]
#         label = self.labels[idx]
#         return image, label





# train_ds = DatasetFmri(image_dictionary = train_dict)
# test_ds = DatasetFmri(image_dictionary = test_dict)

In [138]:
# np.array([np.array([int(x) for x in train_dict['labels']])])

In [139]:
from torch.utils.data import TensorDataset

x_train = torch.from_numpy(train_dict['images'])
y_train = torch.from_numpy(np.array([int(x) for x in train_dict['labels']]))

train_ds = TensorDataset(x_train, y_train)
# test_ds = TensorDataset(torch.from_numpy(test_dict['images']), torch.from_numpy(test_dict['labels']))

In [140]:
train_dl = DataLoader(train_ds, batch_size=bs)

In [141]:
# Note: we can tune hyperparameters here
def get_dataloader(train_ds, test_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs),#, shuffle=True),
        DataLoader(test_ds, batch_size=bs * 2)
    )

In [142]:
bs = 6

train_dl, test_dl = get_dataloader(train_ds, test_ds, bs)

In [143]:
train_features, train_labels = next(iter(train_dl))

## Practice Model

In [151]:
class Fmri_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv3d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv3d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool3d(xb, 4)
        return xb.view(-1, xb.size(1))

In [152]:
model = Fmri_CNN()

epochs = 5 #120
learning_rate = 0.001
loss_funct = F.cross_entropy

opt = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        loss_batch(model, loss_func, xb, yb, opt)

    model.eval()
    # with torch.no_grad():
    #     losses, nums = zip(
    #         *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
    #     )
    # val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

    # print(epoch, val_loss)

RuntimeError: ignored

## Build Model

In [148]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    #Conv1
    self.conv1 = nn.Conv3d(in_channels = 1, 
                           out_channels = 32, 
                           kernel_size = (1,1,1), 
                           stride = (1,1,1)
                           )
    self.bn1 = nn.BatchNorm3d(32)
    self.conv2 = nn.Conv3d(in_channels = 32, 
                           out_channels = 64, 
                           kernel_size = (7,7,7),
                           stride = (2,2,2)
                           )
    self.bn2 = nn.BatchNorm3d(64)
    self.conv3 = nn.Conv3d(in_channels = 64, 
                           out_channels = 64, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn3 = nn.BatchNorm3d(64)
    self.conv4 = nn.Conv3d(in_channels = 64, 
                           out_channels = 128, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn4 = nn.BatchNorm3d(128) 
    self.pool1 = nn.AdaptiveAvgPool3d((1,1,1)) #Global Average Pool, takes the average over last two dimensions to flatten 
  
                                                             
    # Fully connected layer
    self.fc1 = nn.Linear(128,64) # need to find out the size where AdaptiveAvgPool 
    self.fc2 = nn.Linear(64, 2) # left with 2 for the two classes                     



  def forward(self, xb):
    xb = F.relu(self.conv1((xb)))
    xb = self.bn1(xb)
    xb = self.bn2(F.relu(self.conv2((xb))))
    xb = self.bn3(F.relu(self.conv3((xb))))
    xb = self.pool1(self.bn4(F.relu(self.conv4((xb)))))
    xb = self.fc1(xb)
    xb = F.softmax(self.fc2(xb))
    return xb      






In [149]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
print("First model training on GPU")
print(model)

# Initialize other parameters
epochs = 5 #120
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(reduction="mean")
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

First model training on GPU
ConvNet(
  (conv1): Conv3d(1, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv3d(32, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2))
  (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2))
  (bn3): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2))
  (bn4): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): AdaptiveAvgPool3d(output_size=(1, 1, 1))
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)


In [150]:
loss_func = F.cross_entropy

for epoch in range(epochs):
  model.train()
  for xb, yb in train_dl:
    pred = model(xb)
    loss = loss_func(pred, yb)
    print('Loss', loss)

    loss.backward()
    opt.step()
    opt.zero_grad()

  model.eval()

  print(epoch, valid_loss / len(valid_dl))

RuntimeError: ignored

## Training

In [106]:
loss_func = F.cross_entropy

for epoch in range(epochs):
  model.train()
  for xb, yb in train_dl:
    pred = model(xb)
    loss = loss_func(pred, yb)
    print('Loss', loss)

    loss.backward()
    opt.step()
    opt.zero_grad()

  model.eval()
  with torch.no_grad():
    valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)

  print(epoch, valid_loss / len(valid_dl))

RuntimeError: ignored

In [ ]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
print("First model training on GPU")
print(model)

# Initialize other parameters
epochs = 5 #120
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

fit(epochs, model, )

In [20]:
accuracy_stats = {
    'train': [],
    'val': []
  }

print(accuracy_stats)

loss_stats = {
    'train': [],
    'val': []
    }
print(loss_stats)



def train_val_model(epochs):
  for epoch in range(1, epochs + 1):

    # TRAINING *****************************************************************

    train_epoch_loss = 0
    train_epoch_acc = 0

    # set model in training mode 
    model.train()
    print('\nEpoch$ : %d'%epoch)
    for x_train_batch, y_train_batch in tqdm(train_loader):
      x_train_batch = x_train_batch.to(device)#(float).to(device) # for GPU support
      y_train_batch = y_train_batch.to(device) 

      #print(x_train_batch.shape)

      # sets gradients to 0 to prevent interference with previous epoch
      optimizer.zero_grad()
    
      # Forward pass through NN
      y_train_pred = model(x_train_batch)#.to(float)
      train_loss = criterion(y_train_pred, y_train_batch)
      train_acc = accuracy(y_train_pred, y_train_batch)

      # Backward pass, updating weights
      train_loss.backward()
      optimizer.step()

      # Statistics
      train_epoch_loss += train_loss.item()
      train_epoch_acc += train_acc.item()


    # VALIDATION****************************************************************   
    
    with torch.set_grad_enabled(False):
      val_epoch_loss = 0
      val_epoch_acc = 0

      model.eval()
      for x_val_batch, y_val_batch in tqdm(validate_loader):
      
        x_val_batch =  x_val_batch.to(device)#.to(float)
        y_val_batch = y_val_batch.to(device)
            
        # Forward pass
        y_val_pred = model(x_val_batch)#.to(float)   
        val_loss = criterion(y_val_pred, y_val_batch)
        val_acc = accuracy(y_val_pred, y_val_batch)
            
        val_epoch_loss += val_loss.item()
        val_epoch_acc += val_acc.item()

    # Prevent plateauing validation loss 
    #scheduler.step(val_epoch_loss/len(validate_loader))

        
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(validate_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(validate_loader))
                              
    
    print(f'Epoch {epoch+0:03}: Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(validate_loader):.5f}') 
    print(f'Train Acc: {train_epoch_acc/len(train_loader):.3f} | Val Acc: {val_epoch_acc/len(validate_loader):.3f}')

      




def accuracy(y_pred, y_test):
  # Calculating model accuracy at each epoch 
  y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
  _, y_pred_prob = torch.max(y_pred_softmax, dim = 1)
  correct_pred = (y_pred_prob == y_test).float()
  acc = correct_pred.sum() / len(correct_pred)
  acc = torch.round(acc * 100)

  return acc



     





if __name__ == '__main__':
  train_val_model(epochs)

{'train': [], 'val': []}
{'train': [], 'val': []}

Epoch$ : 1


NameError: ignored